## Disclaimer
* [こちらの素晴らしい記事](https://rstudio-pubs-static.s3.amazonaws.com/192092_6a8bac32eac24d7ca3a4791a80e7f3d3.html)をコピペしただけです。

## 問題設定

In [1]:
library(dplyr)
library(tidyr)
library(broom)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [3]:
df <- read.csv("q_data_x.csv")

## 分析1：岩波DS3解説の再現

### 平均値の比較

In [4]:
df %>%
  group_by(cm_dummy) %>% 
  summarise(mean(gamedummy), mean(gamecount), mean(gamesecond)) %>% 
  knitr::kable(digits=3)



| cm_dummy| mean(gamedummy)| mean(gamecount)| mean(gamesecond)|
|--------:|---------------:|---------------:|----------------:|
|        0|           0.073|          10.048|         3107.706|
|        1|           0.075|           8.564|         2478.066|

### セレクションバイアス

In [5]:
inner_join(
  df %>% 
    group_by(cm_dummy) %>% 
    summarise_each(funs(mean), -starts_with("game")) %>% 
    gather(var, mean, -cm_dummy) %>%
    spread(cm_dummy, mean, sep="_mean_"),
  df %>% 
    group_by(cm_dummy) %>% 
    summarise_each(funs(sd), -starts_with("game")) %>% 
    gather(var, sd, -cm_dummy) %>%
    spread(cm_dummy, sd, sep="_sd_"),
  by="var"
) %>% select(var, ends_with("0"), ends_with("1")) %>% 
knitr::kable(digits=3)

Warning message:
“funs() is soft deprecated as of dplyr 0.8.0
please use list() instead

  # Before:
  funs(name = f(.))

  # After: 
  list(name = ~ f(.))
This warning is displayed once per session.”



|var             | cm_dummy_mean_0| cm_dummy_sd_0| cm_dummy_mean_1| cm_dummy_sd_1|
|:---------------|---------------:|-------------:|---------------:|-------------:|
|age             |          40.187|        10.636|          41.767|        10.148|
|area_kanto      |           0.063|         0.243|           0.131|         0.337|
|area_keihan     |           0.509|         0.500|           0.701|         0.458|
|area_keihanshin |           0.303|         0.460|           0.075|         0.263|
|area_tokai      |           0.124|         0.330|           0.093|         0.291|
|child_dummy     |           0.422|         0.494|           0.424|         0.494|
|F1              |           0.138|         0.345|           0.113|         0.317|
|F2              |           0.136|         0.343|           0.226|         0.418|
|F3              |           0.050|         0.218|           0.055|         0.229|
|fam_str_dummy1  |           0.150|         0.358|           0.145|         0.352|
|f

### 平均処置効果の推定

In [6]:
# 傾向スコア推定
ps_model <- glm(
  cm_dummy ~ TVwatch_day + age + sex + marry_dummy + child_dummy +
    inc + pmoney + area_kanto +area_tokai + area_keihanshin +
    job_dummy1 + job_dummy2 + job_dummy3 + job_dummy4 + job_dummy5 +
    job_dummy6 + job_dummy7  + fam_str_dummy1 + fam_str_dummy2 +
    fam_str_dummy3 + fam_str_dummy4,
  data=df, family=binomial()
)
df$ps <- ps_model$fitted.values

# 傾向スコアの逆数の重み
## ATE
df$ipw_ate  <- with(
  df, ifelse(cm_dummy == 1, 1 / ps,  1 / (1 - ps))
)
## ATT
df$ipw_att  <- with(
  df, ifelse(cm_dummy == 1, 1, ps / (1 - ps))
)

# IPW推定値
ipwe_models <- list(
  ## ATE
  ATE_gamedummy  = lm(gamedummy  ~ cm_dummy + I(1-cm_dummy) + 0, weights=ipw_ate, data=df),
  ATE_gamecount  = lm(gamecount  ~ cm_dummy + I(1-cm_dummy) + 0, weights=ipw_ate, data=df),
  ATE_gamesecond = lm(gamesecond ~ cm_dummy + I(1-cm_dummy) + 0, weights=ipw_ate, data=df),
  ## ATT
  ATT_gamedummy  = lm(gamedummy  ~ cm_dummy + I(1-cm_dummy) + 0, weights=ipw_att, data=df),
  ATT_gamecount  = lm(gamecount  ~ cm_dummy + I(1-cm_dummy) + 0, weights=ipw_att, data=df),
  ATT_gamesecond = lm(gamesecond ~ cm_dummy + I(1-cm_dummy) + 0, weights=ipw_att, data=df)
)
lapply(ipwe_models, tidy) %>%
  bind_rows(.id="outcome")%>%
  select(outcome, term, estimate) %>% 
  spread(term, estimate) %>%
  mutate(causal_effect = cm_dummy - `I(1 - cm_dummy)`) %>% 
  knitr::kable(digits=3)



|outcome        | cm_dummy| I(1 - cm_dummy)| causal_effect|
|:--------------|--------:|---------------:|-------------:|
|ATE_gamecount  |   13.636|           8.317|         5.320|
|ATE_gamedummy  |    0.094|           0.062|         0.032|
|ATE_gamesecond | 4143.330|        2639.412|      1503.918|
|ATT_gamecount  |    8.564|           6.249|         2.315|
|ATT_gamedummy  |    0.075|           0.050|         0.026|
|ATT_gamesecond | 2478.066|        2080.266|       397.800|

### 調整効果の推定

In [7]:
# アプリ利用秒数への調整効果
ipwe_mod_models <- list(
  gamesecond_treated = lm(gamesecond ~ child_dummy + area_kanto +area_tokai +
                             area_keihanshin + T + F1 + F2 + F3 + M1 + M2,
                           weights=ipw_ate, data=df[df$cm_dummy==1,]),
  gamesecond_untreated = lm(gamesecond ~ child_dummy + area_kanto +area_tokai +
                               area_keihanshin + T + F1 + F2 + F3 + M1 + M2,
                             weights=ipw_ate, data=df[df$cm_dummy==0,])
)
lapply(ipwe_mod_models, tidy) %>%
  bind_rows(.id="outcome")%>%
  knitr::kable(digits=1)



|outcome              |term            | estimate| std.error| statistic| p.value|
|:--------------------|:---------------|--------:|---------:|---------:|-------:|
|gamesecond_treated   |(Intercept)     |    498.6|     819.6|       0.6|     0.5|
|gamesecond_treated   |child_dummy     |  -1938.9|     666.3|      -2.9|     0.0|
|gamesecond_treated   |area_kanto      |  -2729.2|    1083.7|      -2.5|     0.0|
|gamesecond_treated   |area_tokai      |  -3030.9|     996.9|      -3.0|     0.0|
|gamesecond_treated   |area_keihanshin |   7833.9|     807.3|       9.7|     0.0|
|gamesecond_treated   |T               |   6413.3|    2904.3|       2.2|     0.0|
|gamesecond_treated   |F1              |  -1042.6|    1221.1|      -0.9|     0.4|
|gamesecond_treated   |F2              |   1142.6|    1047.0|       1.1|     0.3|
|gamesecond_treated   |F3              |   -265.1|    1539.0|      -0.2|     0.9|
|gamesecond_treated   |M1              |   1247.3|    1122.2|       1.1|     0.3|
|gamesecond_tr

## 分析2：本に載っていないことを試す

### 傾向スコアによる層別解析

In [8]:
# 層別する（層ごとのデータ数が等しくなるようにする）
df$strata5  <- cut(df$ps, c(0, quantile(df$ps, seq(0,1,0.2))[2:5], 1))
df$strata10 <- cut(df$ps, c(0, quantile(df$ps, seq(0, 1, 0.1))[2:10], 1))
df$strata20 <- cut(df$ps, c(0, quantile(df$ps, seq(0, 1, 0.05))[2:20], 1))

# 平均因果効果
te_strata <- list()
for(strata in c("strata5", "strata10", "strata20")) {
  te_strata[[strata]] <- df %>% 
    group_by_(strata) %>% 
    summarise(te_gamedummy  = sum(cm_dummy*gamedummy)  / sum(cm_dummy) -
                sum((1-cm_dummy)*gamedummy) / sum(1-cm_dummy),
              te_gamecount  = sum(cm_dummy*gamecount)  / sum(cm_dummy) - 
                sum((1-cm_dummy)*gamecount) / sum(1-cm_dummy),
              te_gamesecond = sum(cm_dummy*gamesecond) / sum(cm_dummy) -
                sum((1-cm_dummy)*gamesecond) / sum(1-cm_dummy),
              n=n()) %>% 
    summarise(ate_gamedummy  = sum(te_gamedummy*n) / sum(n),
              ate_gamecount  = sum(te_gamecount*n) / sum(n),
              ate_gamesecond = sum(te_gamesecond*n) / sum(n))
}
knitr::kable(te_strata$strata5,  digits=3)

Warning message:
“group_by_() is deprecated. 
Please use group_by() instead

The 'programming' vignette or the tidyeval book can help you
to program with group_by() : https://tidyeval.tidyverse.org
This warning is displayed once per session.”



| ate_gamedummy| ate_gamecount| ate_gamesecond|
|-------------:|-------------:|--------------:|
|         0.022|         1.499|         68.263|

In [9]:
knitr::kable(te_strata$strata10, digits=3)



| ate_gamedummy| ate_gamecount| ate_gamesecond|
|-------------:|-------------:|--------------:|
|          0.03|         4.564|       1317.082|

In [10]:
knitr::kable(te_strata$strata20, digits=3)



| ate_gamedummy| ate_gamecount| ate_gamesecond|
|-------------:|-------------:|--------------:|
|         0.056|        16.048|       5482.291|

### 傾向スコアによるマッチング

In [11]:
library(Matching)
match_result <- with(df, list(
  gamedummy  = Match(Y=gamedummy,  Tr=cm_dummy, X=ps),
  gamecount  = Match(Y=gamecount,  Tr=cm_dummy, X=ps),
  gamesecond = Match(Y=gamesecond, Tr=cm_dummy, X=ps)
))

summary(match_result$gamedummy)

Loading required package: MASS

Attaching package: ‘MASS’

The following object is masked from ‘package:dplyr’:

    select

## 
##  Matching (Version 4.9-6, Build Date: 2019-04-07)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##




Estimate...  0.020597 
AI SE......  0.0093214 
T-stat.....  2.2097 
p.val......  0.027129 

Original number of observations..............  10000 
Original number of treated obs...............  4144 
Matched number of observations...............  4144 
Matched number of observations  (unweighted).  53955 



In [12]:
summary(match_result$gamecount)


Estimate...  1.6133 
AI SE......  1.7147 
T-stat.....  0.94086 
p.val......  0.34678 

Original number of observations..............  10000 
Original number of treated obs...............  4144 
Matched number of observations...............  4144 
Matched number of observations  (unweighted).  53955 



In [13]:
summary(match_result$gamesecond)


Estimate...  179.44 
AI SE......  551.51 
T-stat.....  0.32535 
p.val......  0.74491 

Original number of observations..............  10000 
Original number of treated obs...............  4144 
Matched number of observations...............  4144 
Matched number of observations  (unweighted).  53955 



In [14]:
# CMを見てもほとんどの人はゲームをやらない
data.frame(
  treated   = quantile(df$gamecount[df$cm_dummy==0], seq(0,1,0.1)),
  untreated = quantile(df$gamecount[df$cm_dummy==1], seq(0,1,0.1))
) %>% t() %>% knitr::kable()



|          | 0%| 10%| 20%| 30%| 40%| 50%| 60%| 70%| 80%| 90%| 100%|
|:---------|--:|---:|---:|---:|---:|---:|---:|---:|---:|---:|----:|
|treated   |  0|   0|   0|   0|   0|   0|   0|   0|   0|   0|  765|
|untreated |  0|   0|   0|   0|   0|   0|   0|   0|   0|   0|  688|

### 傾向スコアによるカーネルマッチング

In [15]:
# バンド幅（赤本に載っているおまじないの値）
bw1 <- 1.06 * nrow(df[df$cm_dummy==1,])^(-0.2) * sd(df$ps[df$cm_dummy==1])
bw0 <- 1.06 * nrow(df[df$cm_dummy==0,])^(-0.2) * sd(df$ps[df$cm_dummy==0])

km_result <- list()
for(col in c("gamedummy", "gamecount", "gamesecond")) {
  # カーネル回帰
  ## CM非接触群の、仮にCMに接触していた場合の結果変数
  y1_est <- ksmooth(x=df$ps[df$cm_dummy==1], y=df[df$cm_dummy==1, col],
                    x.points=df$ps[df$cm_dummy==0], bandwidth=bw1, kernel="normal")
  ## CM接触群の、仮にCMに接触していなかった場合の結果変数
  y0_est <- ksmooth(x=df$ps[df$cm_dummy==0], y=df[df$cm_dummy==0, col],
                    x.points=df$ps[df$cm_dummy==1], bandwidth=bw0, kernel="normal")

  # 平均因果効果
  ate <- structure(
    mean(c(df[df$cm_dummy==1, col], y1_est$y) - c(y0_est$y, df[df$cm_dummy==0, col])),
    names=col
  )
  km_result[col] <- ate
}

knitr::kable(as.data.frame(km_result), digits=3)



| gamedummy| gamecount| gamesecond|
|---------:|---------:|----------:|
|     0.038|     9.251|    2838.06|

### 傾向スコアをロジスティック回帰以外の方法で推定する

In [16]:
library("randomForest")
library("gbm")

# ランダムフォレスト（共変量はロジスティック回帰と同じ，パラメータは適当）
ps_rf <- randomForest(as.factor(cm_dummy) ~ TVwatch_day + age + sex + marry_dummy + child_dummy +
                        inc + pmoney + area_kanto +area_tokai + area_keihanshin +
                        job_dummy1 + job_dummy2 + job_dummy3 + job_dummy4 + job_dummy5 +
                        job_dummy6 + job_dummy7  + fam_str_dummy1 + fam_str_dummy2 +
                        fam_str_dummy3 + fam_str_dummy4,
                      data = df)
ps_rf$fitted.values <- ps_rf$votes[,"1"]

# GBM（共変量はロジスティック回帰と同じ，パラメータは適当）
ps_gbm <- gbm(formula = cm_dummy ~ TVwatch_day + age + sex + marry_dummy + child_dummy +
                inc + pmoney + area_kanto +area_tokai + area_keihanshin +
                job_dummy1 + job_dummy2 + job_dummy3 + job_dummy4 + job_dummy5 +
                job_dummy6 + job_dummy7  + fam_str_dummy1 + fam_str_dummy2 +
                fam_str_dummy3 + fam_str_dummy4,
              data = df, distribution = "bernoulli", n.trees=1000, interaction.depth = 5)
ps_gbm$fitted.values <- 1 / (1 + exp(-ps_gbm$fit))

# 傾向スコア推定のAUC（c統計量）確認
c(pROC::auc(df$cm_dummy, ps_model$fitted.values),
  pROC::auc(df$cm_dummy, ps_rf$fitted.values),
  pROC::auc(df$cm_dummy, ps_gbm$fitted.values))

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:dplyr’:

    combine

Loaded gbm 2.1.5
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases


[1] 0.7917029 0.9938445 0.9998012

In [17]:
# IPW推定量
df$ps_rf <- ps_rf$fitted.values
df$ipw_rf  <- with(df, ifelse(cm_dummy == 1, 1 / ps_rf,  1 / (1 - ps_rf)))
df$ps_gbm <- ps_gbm$fitted.values
df$ipw_gbm  <- with(df, ifelse(cm_dummy == 1, 1 / ps_gbm,  1 / (1 - ps_gbm)))

ipwe_models2 <- list(
  lr_gamedummy  = lm(gamedummy  ~ cm_dummy + I(1-cm_dummy) + 0, weights=ipw_ate, data=df),
  lr_gamecount  = lm(gamecount  ~ cm_dummy + I(1-cm_dummy) + 0, weights=ipw_ate, data=df),
  lr_gamesecond = lm(gamesecond ~ cm_dummy + I(1-cm_dummy) + 0, weights=ipw_ate, data=df),
  rf_gamedummy   = lm(gamedummy  ~ cm_dummy + I(1-cm_dummy) + 0, weights=ipw_rf,  data=df),
  rf_gamecount   = lm(gamecount  ~ cm_dummy + I(1-cm_dummy) + 0, weights=ipw_rf,  data=df),
  rf_gamesecond  = lm(gamesecond ~ cm_dummy + I(1-cm_dummy) + 0, weights=ipw_rf,  data=df),
  gbm_gamedummy  = lm(gamedummy  ~ cm_dummy + I(1-cm_dummy) + 0, weights=ipw_gbm, data=df),
  gbm_gamecount  = lm(gamecount  ~ cm_dummy + I(1-cm_dummy) + 0, weights=ipw_gbm, data=df),
  gbm_gamesecond = lm(gamesecond ~ cm_dummy + I(1-cm_dummy) + 0, weights=ipw_gbm, data=df)
)
lapply(ipwe_models2, tidy) %>%
  bind_rows(.id="outcome") %>%
  dplyr::select(outcome, term, estimate) %>% 
  spread(term, estimate) %>%
  mutate(causal_effect = cm_dummy - `I(1 - cm_dummy)`) %>% 
  knitr::kable(digits=3)



|outcome        | cm_dummy| I(1 - cm_dummy)| causal_effect|
|:--------------|--------:|---------------:|-------------:|
|gbm_gamecount  |    9.496|          10.003|        -0.507|
|gbm_gamedummy  |    0.077|           0.073|         0.004|
|gbm_gamesecond | 2684.439|        3096.247|      -411.807|
|lr_gamecount   |   13.636|           8.317|         5.320|
|lr_gamedummy   |    0.094|           0.062|         0.032|
|lr_gamesecond  | 4143.330|        2639.412|      1503.918|
|rf_gamecount   |   13.699|           9.991|         3.708|
|rf_gamedummy   |    0.089|           0.072|         0.017|
|rf_gamesecond  | 3623.848|        3126.687|       497.161|

In [18]:
# 傾向スコア推定値のプロット
caret::calibration(as.factor(cm_dummy) ~ ps+ps_rf+ps_gbm, class="1", data=df) %>%
  lattice::xyplot(auto.key = list(columns = 2))

### 決定技による因果効果の異質性の推定

In [19]:
#devtools::install_github("susanathey/causalTree")
library(causalTree)
library(rpart.plot)

ct_gamesecond <- causalTree(
  gamesecond ~ child_dummy + area_kanto +area_tokai + area_keihanshin + area_keihan +
    T + F1 + F2 + F3 + M1 + M2 + M3,
  data = df[,1:35], treatment = df$cm_dummy, weights = df$ipw_ate,
  split.Rule = "CT", cv.option = "CT", 
  split.Honest = TRUE, cv.Honest = TRUE, split.Bucket = FALSE, 
  xval = 5, cp = 0, minsize = 100
)

cp_opt <- ct_gamesecond$cptable[,1][which.min(ct_gamesecond$cptable[,4])]
ct_gamesecond_opt <- prune(ct_gamesecond, cp_opt)
rpart.plot(ct_gamesecond_opt, extra=101, nn=TRUE)

ERROR: Error in library(causalTree): there is no package called ‘causalTree’
